In [14]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from collections import defaultdict
import os
import re

In [3]:
# os.cpu_count()

In [4]:
foundFile = False
while not foundFile:
    path = input('Enter your filepath to the data (.csv only, enter "exit" to end): ')
    if path == 'exit':
        break
    elif path[-4:] != '.csv':
        print("That's not a csv file!")
        continue
    try:
        df = pd.read_csv(path)
        foundFile = True
    except FileNotFoundError:
        print('File not found. Enter your filepath again.')
print('============ Column Names ============')
print(df.columns)
print('============ DataFrame ============')
print(df)

============ Column Names ============
Index(['student_id', 'age', 'gender', 'course', 'study_hours',
       'class_attendance', 'internet_access', 'sleep_hours', 'sleep_quality',
       'study_method', 'facility_rating', 'exam_difficulty', 'exam_score'],
      dtype='object')
============ DataFrame ============
       student_id  age  gender   course  study_hours  class_attendance  \
0               1   17    male  diploma         2.78              92.9   
1               2   23   other      bca         3.37              64.8   
2               3   22    male     b.sc         7.88              76.8   
3               4   20   other  diploma         0.67              48.4   
4               5   20  female  diploma         0.89              71.6   
...           ...  ...     ...      ...          ...               ...   
19995       19997   18   other      bba         6.50              71.3   
19996       19998   18    male    b.com         3.71              41.6   
19997       19999   

In [5]:
cols = df.columns[df.dtypes.eq('object')]
for c in cols:
    df[c] = df[c].astype('category').cat.codes


d = input('Enter the column(s) to drop (Enter "#NONE" if none): ')
li = d.split(' ')
print(li)
if li != ['#NONE']:
    for c in li:
        try:
            df.drop(columns=[c])
        except KeyError:
            print('Drop column failed, the column was', c)

scaler = StandardScaler()
scaled = scaler.fit_transform(df)

print(scaled)

['#NONE']
[[-1.73196262 -1.52044246 -0.00901182 ... -0.00966555 -0.21348541
  -0.19109485]
 [-1.73178942  1.10606684  1.2170862  ...  1.21382199  0.9290643
  -0.40793407]
 [-1.73161622  0.66831529 -0.00901182 ... -1.23315309  0.9290643
   1.46957626]
 ...
 [ 1.73178783 -0.64493936  1.2170862  ... -0.00966555 -1.35603512
   0.10507579]
 [ 1.73196103 -0.64493936 -0.00901182 ...  1.21382199  0.9290643
   0.87194621]
 [ 1.73213424 -0.20718781 -0.00901182 ...  1.21382199  0.9290643
   0.44884529]]


In [ ]:
while True:
    response = input('Enter the response variable: ')
    try:
        x = df.drop(columns=[response])
        y = df[response]
        
        break
    except KeyError:
        print('That column does not exist, please try again.')
        continue

while True:
    methods = input('Choose the regression methods you want to add (1. Linear regression 2. Polynomial 3. Ridge 4. Lasso 5. Logistic 6. Poisson 7. Quantile)')
    methods_list = methods.split(' ')
    have_methods = [False for _ in range(8)]
    have = False
    for m in methods_list:
        try:
            if int(m) >= 1 and int(m) <= 7:
                have_methods[int(m)-1] = True
                have = True
        except ValueError:
            print('Error!!!! The input ' + m + ' is not valid.')
    if have:
        break

cpu_cores = os.cpu_count()

linear_parems = defaultdict(list)


# fit_intercept: Whether to calculate the intercept for this model. If set to False, no intercept will be used in calculations (i.e. data is expected to be centered).
# tol: The precision of the solution (coef_) is determined by tol which specifies a different convergence criterion for the lsqr solver.
# n_jobs: how much cpu cores will be used
# positive: force the coeffients to be positive or not
valid_li=[]
def do_linear():
    params_to_test = input('Enter the parameters you want to test. (1. fit_intercept 2. tol 3. n_jobs 4. positive')
    parems_li = params_to_test.split(' ')
    # filiter invalid inputs list like [fdjhdsafl, 1, 2, dfakjfk, 3, 100]
    global valid_li
    valid_li = [p for p in parems_li if re.match(pattern=r'^[1-4]$',string=p)]
    for p in parems_li:
        # fit_intercept
        if p == '1':
            fit_inter = input('Enter the values you want to try on '+'fit_intercept'+' (T/F)')
            fit_list = set(fit_inter.split(' '))
            if "T" in fit_list:
                linear_parems['fit_intercept'].append(True)
            elif "F" in fit_list:
                linear_parems['fit_intercept'].append(False)
        # tol
        elif p == '2':
            tol = set(input('Enter the values you want to try on '+'tol'+' (float numbers)').split(' '))
            for t in tol:
                try:
                    if float(t) < 1:
                        linear_parems['tol'].append(float(t))
                    else:
                        print(t +'is too big!!!!!!!!!!!!!!')
                except ValueError:
                    print(t +'was an invalid input!!!!!!!!!!!!!!')
        # n_jobs
        elif p == '3':
            nj = set(input('Enter the values you want to try on '+'n_jobs'+' (integer numbers)').split(' '))
            for n in nj:
                try:
                    if int(n) < 0:
                        if abs(int(n)) < cpu_cores:
                            linear_parems['n_jobs'].append(int(n))
                    elif int(n) >0:
                        if abs(int(n)) <= cpu_cores:
                            linear_parems['n_jobs'].append(int(n))
                except ValueError:
                    print(n +'was an invalid input!!!!!!!!!!!!!!')
        # positive
        elif p == '4':
            pos = set(input('Enter the values you want to try on '+'positive'+' (T/F)').split(' '))
            if "T" in pos:
                linear_parems['positive'].append(True)
            elif "F" in pos:
                linear_parems['positive'].append(False)


poly_parem = defaultdict(list)

# interaction_only == True --> a1,a2 degree=2 --> won't generate a1^2 and a2^2 (force the inputs to interact)
# include_bias: add bias(constant) into our model or not
# order: we have x1,x2,x1^2,x2^2 -> 'C':x1,x1^2,x2,x2^2 | 'F':x1,x2,x1^2,x2^2

def do_poly():
    params_to_test = input('Enter the parameters you want to test. (1. degree 2. interaction_only 3. include_bias 4. order').split(' ')
    for p in params_to_test:
        # degree
        # 4,6 7
        if p == '1':
            deg = set(input('Enter the degree you want (seperate min max with commas)').split(' '))
            for d in deg:
                if re.search(r'^[0-9]+,[0-9]+$',d):
                    ds = d.split(',')
                    try:
                        if int(ds[0]) < int(ds[1]):
                            poly_parem['degree'].append((ds[0],ds[1]))
                        else:
                            print(d,' is not a valid input!!!!!!!!')
                    except ValueError:
                        print(d,' is not a valid input!!!!!!!!')
                elif re.search(r'^[0-9]+$',d):
                    try:
                        poly_parem['degree'].append(int(d))
                    except ValueError:
                        print(d,' is not a valid input!!!!!!!!')
        # interaction_only
        elif p == '2':
            deg = set(input('Enter the degree you want (T/F)').split(' '))
            for d in deg:
                if d == 'T':
                    poly_parem['interaction_only'].append(True)
                elif d == 'F':
                    poly_parem['interaction_only'].append(False)
                else:
                    print(d,' is not a valid input!!!!!!!!')
        # include_bias
        elif p == '3':
            deg = set(input('Do you want to add a bias column into the model (T/F)').split(' '))
            for d in deg:
                if d == 'T':
                    poly_parem['include_bias'].append(True)
                elif d == 'F':
                    poly_parem['include_bias'].append(False)
                else:
                    print(d,' is not a valid input!!!!!!!!')
        #order
        elif p == '4':
            orders = set(input('Enter the order you want to try (C/F)').split(' '))
            for o in orders:
                if o == 'C':
                    poly_parem['order'].append('C')
                elif o == 'F':
                    poly_parem['order'].append('F')

# ridge regression

ridge_parems = {}
# alpha: regularlization strength[0,np.inf)


def do_ridge():
    pars = set(input('Enter the (alpha,fit_intercept,max_iter,tol,solver)').split(' '))
    for p in pars:
        if p == '1':
            alphas = set(input('Enter the alpha you want to try [0,np.inf)').split(' '))
            for a in alphas:
                try:
                    if a == 'np.inf':
                        poly_parem['alpha'].append(np.inf)
                    poly_parem['alpha'].append(float(a))
                except ValueError:
                    print(a,'was an invalid input!!!!!!!!!!!!!!!!!!!!')
        
for i,v in enumerate(have_methods):
    if i == 0 and v:
        do_linear()
    elif i == 1 and v:
        do_poly()
    elif i == 2 and v:
        do_ridge()
print(linear_parems)

Error!!!! The input dfansjfnad is not valid.
Error!!!! The input  is not valid.
Error!!!! The input 34n is not valid.
Error!!!! The input fdajndfjkndfjkanfjladf is not valid.
Error!!!! The input nla is not valid.
defaultdict(<class 'list'>, {'n_jobs': [2]})


In [7]:
valid_li

['3']

In [ ]:
best_fits = {}

def grid_searching(reg_type,reg_parems,reg_name):
    global best_fits
    clf = GridSearchCV(reg_type,reg_parems)
    clf.fit(x,y)
    best_fits[reg_name] = [clf.best_score_,clf.best_params_]

for v in valid_li:
    if v=='1':
        grid_searching(LinearRegression(),linear_parems,'Linear Regression')
    elif v == '2':
        grid_searching(PolynomialFeatures(),poly_parem,'Polynomial Regression')

#   type  best_score  best_parem
# {'linear':[0.59,{'C':np.inf}]}


for best_k,best_v in best_fits.items():
    print(best_k,'with the paremeters of ', str(best_v[1]),'has a score of',best_v[0])


In [11]:
di = {'jfdioajoidfsa':123,'adfjiodasjfi':25}
print(str(di))

{'jfdioajoidfsa': 123, 'adfjiodasjfi': 25}


In [13]:
clf.fit(x,y)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",LinearRegression()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","defaultdict(<...': [2, 3, 1]})"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",None
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate parameter i

In [14]:
clf.best_params_

{'n_jobs': 2}

linear with the paremeters of  {'C': 1, 'dfmndfas': 23} has a score of 0.59
